In [0]:
import pandas as pd
import json

In [2]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/CSE 245 Project


In [3]:
!ls Data

'Data Exploration.ipynb'
 data_fixed_train.json
 data_sample_100.json
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_3999.csv
 data_sample_fixed_processed_model2_4999.csv
 data_sample_fixed_processed_model2_5999.csv
 data_sample_fixed_processed_model2_6999.csv
 data_sample_fixed_processed_model2_7999.csv
 data_sample_fixed_processed_model2_8999.csv
 data_sample_fixed_processed_model2_9999.csv
 data_sample_fixed_processed_model2.csv
 data_sample_fixed_processed_model2_intermediate.csv
 data_sample_fixed_processed_model3.csv
 data_sample_fixed_processed_model4.csv
 fixed_train_516.csv
'informative words.ipynb'
 Raw
 valid.csv


In [48]:
with open ("Data/fixed_test.csv","w") as outfile:
  with open("Data/Raw/test.csv","r") as infile:
    line = infile.readline()
    outfile.write(line)
    print(line)
    error_cnt = 0
    cnt = 1
    #print(len(line.split(",")))
    while line:
        if len(line.split(",")) != 8:
          print(line.split(",")[7])
          error_cnt += 1
          print("Line {}: {}".format(cnt, line.strip()))
        line = infile.readline()
        line = line.replace(",,",",").replace(",<UNIGRAM>,",",<UNIGRAM>").replace(
            ",<HI>,",",<HI>").replace(",<HI> <UNIGRAM>,",",<HI> <UNIGRAM>").replace(
                ",<UNIGRAM> <NUMERAL>,",",<UNIGRAM> <NUMERAL>").replace(
                    ",<IRREGULAR_COLON_FORMAT>,",",<IRREGULAR_COLON_FORMAT>").replace(",<POLITICAL>,",",<POLITICAL>")
        outfile.write(line)
        cnt += 1
print(cnt, error_cnt)

conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags

10975 0


In [0]:
train_df = pd.read_csv('Data/fixed_valid.csv', sep=",")

In [58]:
train_df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:3_conv:6,1,terrified,Today_comma_as i was leaving for work in the m...,6,Today_comma_as i was leaving for work in the m...,4|5|5_5|5|5,NaN
1,hit:3_conv:6,2,terrified,Today_comma_as i was leaving for work in the m...,7,Are you fine now?,4|5|5_5|5|5,It's amazing how many feelings and memories me...
2,hit:3_conv:6,3,terrified,Today_comma_as i was leaving for work in the m...,6,Yeah_comma_i'm doing alright now_comma_ but wi...,4|5|5_5|5|5,NaN
3,hit:3_conv:6,4,terrified,Today_comma_as i was leaving for work in the m...,7,Cool :) Is your car damaged a lot?,4|5|5_5|5|5,<IRREGULAR_COLON_FORMAT>I've actually never go...
4,hit:3_conv:6,5,terrified,Today_comma_as i was leaving for work in the m...,6,The car was badly damaged_comma_i veered outsi...,4|5|5_5|5|5,NaN
...,...,...,...,...,...,...,...,...
12025,hit:12361_conv:24722,4,prepared,One time I studied all night for my final exam!,46,tha is really cool what was your grade,4|4|5_4|4|3,must be nice to have someone that loves you|Th...
12026,hit:12392_conv:24785,1,furious,One of my coworkers has been arguing with his ...,791,One of my coworkers has been arguing with his ...,4|5|5_5|5|5,NaN
12027,hit:12392_conv:24785,2,furious,One of my coworkers has been arguing with his ...,829,What are they arguing about?,4|5|5_5|5|5,Well that is great and sad all at the same tim...
12028,hit:12392_conv:24785,3,furious,One of my coworkers has been arguing with his ...,791,Everything and anything. It's annoying_comma_ ...,4|5|5_5|5|5,NaN


In [59]:
train_df[['hit','conv_id']] = train_df.conv_id.str.split("_",expand=True)
train_df['conv_id'] = train_df['conv_id'].map(lambda x: x.lstrip('conv_id:'))
train_df['hit'] = train_df['hit'].map(lambda x: x.lstrip('hit:'))
train_df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,hit
0,6,1,terrified,Today_comma_as i was leaving for work in the m...,6,Today_comma_as i was leaving for work in the m...,4|5|5_5|5|5,NaN,3
1,6,2,terrified,Today_comma_as i was leaving for work in the m...,7,Are you fine now?,4|5|5_5|5|5,It's amazing how many feelings and memories me...,3
2,6,3,terrified,Today_comma_as i was leaving for work in the m...,6,Yeah_comma_i'm doing alright now_comma_ but wi...,4|5|5_5|5|5,NaN,3
3,6,4,terrified,Today_comma_as i was leaving for work in the m...,7,Cool :) Is your car damaged a lot?,4|5|5_5|5|5,<IRREGULAR_COLON_FORMAT>I've actually never go...,3
4,6,5,terrified,Today_comma_as i was leaving for work in the m...,6,The car was badly damaged_comma_i veered outsi...,4|5|5_5|5|5,NaN,3
...,...,...,...,...,...,...,...,...,...
12025,24722,4,prepared,One time I studied all night for my final exam!,46,tha is really cool what was your grade,4|4|5_4|4|3,must be nice to have someone that loves you|Th...,12361
12026,24785,1,furious,One of my coworkers has been arguing with his ...,791,One of my coworkers has been arguing with his ...,4|5|5_5|5|5,NaN,12392
12027,24785,2,furious,One of my coworkers has been arguing with his ...,829,What are they arguing about?,4|5|5_5|5|5,Well that is great and sad all at the same tim...,12392
12028,24785,3,furious,One of my coworkers has been arguing with his ...,791,Everything and anything. It's annoying_comma_ ...,4|5|5_5|5|5,NaN,12392


In [0]:
data = {}
for rownum, row in train_df.iterrows():
    #print(row)
    #print(row[1]['conv_id'])
    conv_id = row['conv_id']
    if conv_id not in data.keys():
        item = {}
        item["prompt"] = row['prompt']
        item["emotion"] = row['context']
        item["dialog"] = []
        item["dialog"].append({"speaker":row['speaker_idx'],'utterance':row['utterance']})
        data[conv_id] = item
    elif row['prompt'] != data[conv_id]["prompt"]:
        print("Prompt changed for line: %s, for conv_id: %s" % (rownum,conv_id))
    else:
        data[conv_id]["dialog"].append({"speaker":row['speaker_idx'],'utterance':row['utterance']})
#print(data)

In [0]:
with open('Data/fixed_valid.json', 'w') as outfile:
    json.dump(data, outfile, indent=4, sort_keys=True)

In [0]:
filter_df = train_df[train_df['context'] == 'anticipating']
pair_df = filter_df[['prompt','utterance']]
pair_df

,prompt,utterance


In [0]:
print(pair_df["prompt"][117])
print(pair_df["utterance"][117])
print(pair_df["prompt"][118])
print(pair_df["utterance"][118])
print(pair_df["prompt"][119])
print(pair_df["utterance"][119])
print(pair_df["prompt"][120])
print(pair_df["utterance"][120])

when I was younger_comma_ my parents picked me up from work with a new car_comma_ which really surprised me_comma_ I was so excited to have my own car!
When I was working my first job_comma_ my parents picked me up in my new car_comma_ I was very surprised_comma_ i thought I was gonna have to buy my own car
when I was younger_comma_ my parents picked me up from work with a new car_comma_ which really surprised me_comma_ I was so excited to have my own car!
wow_comma_ that was so awesome of them!
when I was younger_comma_ my parents picked me up from work with a new car_comma_ which really surprised me_comma_ I was so excited to have my own car!
Yeah it really was_comma_ I am still very thankful of them
when I was younger_comma_ my parents picked me up from work with a new car_comma_ which really surprised me_comma_ I was so excited to have my own car!
I had to buy my first car and it wasn't until I was 21 that I could afford it. 


In [0]:
train_df.groupby('context')['conv_id'].nunique()

context
afraid           634
angry            695
annoyed          667
anticipating     606
anxious          614
apprehensive     462
ashamed          493
caring           527
confident        616
content          576
devastated       569
disappointed     598
disgusted        619
embarrassed      563
excited          750
faithful         377
furious          615
grateful         645
guilty           622
hopeful          619
impressed        621
jealous          585
joyful           602
lonely           641
nostalgic        599
prepared         592
proud            686
sad              667
sentimental      525
surprised       1004
terrified        629
trusting         515
Name: conv_id, dtype: int64

In [0]:
train_df.groupby('prompt').nunique()

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,hit
prompt,,,,,,,,,
I used to be afraid of the dark but now I am not so much_comma_ I had a nightlight for a long time,2,4,1,1,2,7,1,0,2
A driver brushed my car with his car and did not stop to apologize and fix my car,1,4,1,1,2,4,1,0,1
A motorist ride his bike at a extreme speed without any control over it. I enraged at this motorist. He should be punished for his actions.,1,4,1,1,2,4,1,0,1
A nice storm came through here today and totally knocked some piece off my cable dish. The cable is not working like at all_comma_ so I called the company and they said they will have someone out tomorrow to fix it. I feel off the grid right now.,1,4,1,1,2,4,1,0,1
A stitch in time saves nine,1,4,1,1,2,4,1,0,1
...,...,...,...,...,...,...,...,...,...
yesterday I made the bed with new sheets and comforter. Last night I was pretty tired and decided to try out the new bed and watch my shows. Omg_comma_ those sheets were so comfortable I fell asleep in the middle of my show.,1,7,1,1,2,7,1,1,1
yesterday after work i decided to walk to the store to get groceries and it started raining.it rained on me and i was very wet by the time i went got to the store,1,8,1,1,2,8,1,0,1
yesterday at work some guy ran his work truck into our building,1,4,1,1,2,4,1,0,1


In [0]:
train_df.groupby('tags').nunique()

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags,hit
tags,,,,,,,,,
<HI>,165,7,32,165,70,169,50,1,165
<HI> <UNIGRAM>,7,3,5,7,9,5,7,1,7
<IRREGULAR_COLON_FORMAT>,13,3,10,13,9,12,2,1,13
<POLITICAL>,103,6,27,103,38,134,46,1,101
<UNIGRAM>,331,8,32,328,117,204,102,1,323
<UNIGRAM> <NUMERAL>,69,6,27,67,47,25,31,1,69


conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags

84171 0
